# FINVIZ Web Scraper

In [1]:
import requests
import pandas as pd
from datetime import date, datetime
import time
import calendar
from bs4 import BeautifulSoup as bs4
from random import randint
from time import sleep
import math
from IPython.display import clear_output

In [2]:
"""
VTSMX is Vanguard's total stock market fund. I'll be collecting a list of
stocks to scrape data for from the holdings of this fund.
downloaded VTSMX.csv from:
https://institutional.vanguard.com/web/c1/investments/product-details/fund/0085
"""
vtsmx = pd.read_csv('VTSMX.csv')
tickers = vtsmx['Ticker']
tickers.describe()

count     3509
unique    3495
top        ---
freq         8
Name: Ticker, dtype: object

In [3]:
print(tickers[tickers.duplicated(keep=False)])

600                                CZR
788                                CZR
3034                               ---
3087                               ---
3336                               ---
3392                               ---
3394                               ---
3495                               ---
3496                               ---
3497                               ---
3500                               NaN
3501                     Holdings name
3502       United States Treasury Bill
3503       United States Treasury Bill
3504       United States Treasury Bill
3505                               NaN
3506                     Holdings name
3507                               NaN
3508                     Holdings name
3509                               NaN
3510                     Holdings name
3511    Vanguard Market Liquidity Fund
3512    Vanguard Market Liquidity Fund
3513                               NaN
3514                               NaN
Name: Ticker, dtype: obje

In [4]:
tickers = tickers.drop_duplicates(keep = False)
tickers.describe()

count     3490
unique    3490
top       CNDT
freq         1
Name: Ticker, dtype: object

In [5]:
"""
Example GET request headers from FINVIZ.com

Found in network tab of browser dev tools.
"""
# GET /quote.ashx?t=amzn HTTP/1.1
# Host: finviz.com
# Connection: keep-alive
# Cache-Control: max-age=0
# Upgrade-Insecure-Requests: 1
# User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44
# Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
# Sec-Fetch-Site: same-origin
# Sec-Fetch-Mode: navigate
# Sec-Fetch-User: ?1
# Sec-Fetch-Dest: document
# Referer: https://finviz.com/
# Accept-Encoding: gzip, deflate, br
# Accept-Language: en-US,en;q=0.9
# Cookie: pv_date=Thu Sep 03 2020 12:08:33 GMT-0400 (Eastern Daylight Time); pv_count=6



'\nExample GET request headers from FINVIZ.com\n\nFound in network tab of browser dev tools.\n'

In [7]:
# spoofing the GET
pv_count = 1
day_now = calendar.day_abbr[date.today().weekday()]
date_now = date.today().strftime("%b %d %Y")
time_now = datetime.now().strftime("%H:%M:%S")
headers = {
    'Host': 'finviz.com',
    'Connection': 'keep-alive',
    'Cache-Control':'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
   'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://finviz.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cookie': f'pv_date={day_now} {date_now} {time_now} GMT-0400 (Eastern Daylight Time); pv_count={pv_count}',
}
tick = 'MSFT'
response = requests.get(f'https://finviz.com/quote.ashx?t={tick}', headers=headers)
soup = bs4(response.content, 'html.parser')

In [8]:
# first we need to get all the features we'll be collecting
features = []
table = soup.find('table', {"class":"snapshot-table2"})
for row in table.find_all('tr', recursive=False):
    for column in row.find_all('td', class_='snapshot-td2-cp', recursive=False):
        features.append(column.contents[0])

In [9]:
stocks = pd.DataFrame(index = tickers, columns = features)
stocks.head()

,Index,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y,Insider Trans,...,Earnings,Payout,Avg Volume,Price,Recom,SMA20,SMA50,SMA200,Volume,Change
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MSFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AMZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GOOGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
# now to figure out how to add data from the html
i = 0
for row in table.find_all('tr', recursive=False):
    for column in row.find_all('td', class_='snapshot-td2', recursive=False):
        if column.find('b').find('span') != None:
            stocks.loc['MSFT', features[i]] = column.find('b').find('span').contents[0]
        elif column.find('b').find('small') != None:
             stocks.loc['MSFT', features[i]] = column.find('b').find('small').contents[0]
        else:
             stocks.loc['MSFT', features[i]] = column.find('b').contents[0]
        i += 1
stocks.head()

,Index,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y,Insider Trans,...,Earnings,Payout,Avg Volume,Price,Recom,SMA20,SMA50,SMA200,Volume,Change
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MSFT,DJIA S&P500,37.70,5.76,0.10%,7.58B,-4.10%,1627.71B,29.62,13.59%,-5.88%,...,Jul 22 AMC,34.20%,34.38M,217.30,1.70,0.66%,3.35%,21.59%,"57,706,830",-6.19%
AMZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GOOGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Scraping

In [4]:
def populateRow(stocks, tick, features, soup):
    i = 0
    table = soup.find('table', {"class":"snapshot-table2"})
    if not soup.find('table', {"class":"snapshot-table2"}) == None:
        for row in table.find_all('tr', recursive=False):
            for column in row.find_all('td', class_='snapshot-td2', recursive=False):
                if column.find('b').find('span') != None:
                    stocks.loc[tick, features[i]] = column.find('b').find('span').contents[0]
                elif column.find('b').find('small') != None:
                     stocks.loc[tick, features[i]] = column.find('b').find('small').contents[0]
                else:
                     stocks.loc[tick, features[i]] = column.find('b').contents[0]
                i += 1
        return stocks
    else:
        stocks.loc[tick, 'Index'] = "Missing"
        return stocks
        

def makeRequest(tick, pv_count):
    tick = tick.replace('.', '-')
    day_now = calendar.day_abbr[date.today().weekday()]
    date_now = date.today().strftime("%b %d %Y")
    time_now = datetime.now().strftime("%H:%M:%S")
    
    # GET /quote.ashx?t=amzn HTTP/1.1
    # Host: finviz.com
    # Connection: keep-alive
    # Cache-Control: max-age=0
    # Upgrade-Insecure-Requests: 1
    # User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44
    # Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
    # Sec-Fetch-Site: same-origin
    # Sec-Fetch-Mode: navigate
    # Sec-Fetch-User: ?1
    # Sec-Fetch-Dest: document
    # Referer: https://finviz.com/
    # Accept-Encoding: gzip, deflate, br
    # Accept-Language: en-US,en;q=0.9
    # Cookie: pv_date=Thu Sep 03 2020 12:08:33 GMT-0400 (Eastern Daylight Time); pv_count=6

    headers = {
        'Host': 'finviz.com',
        'Connection': 'keep-alive',
        'Cache-Control':'max-age=0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-User': '?1',
        'Sec-Fetch-Dest': 'document',
        'Referer': 'https://finviz.com/',
#         'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cookie': f'screenerUrl=screener.ashx?v=111&f=sec_technology; __cfduid=d26282a3881cb08056c8ddaf5619127961600097084; pv_date={day_now} {date_now} {time_now} GMT-0400 (Eastern Daylight Time); pv_count={pv_count}',
    }
    try:
        response = requests.get(f'https://finviz.com/quote.ashx?t={tick}', headers=headers)
    except requests.exceptions.ConnectionError:
        print(response)
    return bs4(response.content, 'html.parser', from_encoding="utf-8")
    

In [7]:
stocks = pd.read_csv('stocks.csv', index_col=0)
features = list(stocks.columns.values)

In [20]:
global pv_count
# pv_count = 0
i = 1
for tick,_ in stocks.iterrows():
    if isinstance(stocks.loc[tick, 'Index'], float):
        clear_output(wait=True)
        print(f'Requesting {tick} {pv_count}. . . ', end=' ')
        soup = makeRequest(tick, pv_count)        
        print(f'Populating . . . ', end=' ')
        stocks = populateRow(stocks, tick, features, soup)
        pv_count += 1
        i += 1
        if i % 10 == 0:
            print(f'.%SAVING%.', end=' ')
            stocks.to_csv('stocks.csv')
        sec = randint(10,35)#35)
        print(f'Relaxing for {sec} sec . . . ')
        print(stocks.iloc[pv_count - 15:pv_count - 9, 0:6])
        sleep(sec)
print(f'DONE')

Requesting FTBK 3501. . .  Populating . . .  Relaxing for 17 sec . . . 
          Index   P/E EPS (ttm) Insider Own Shs Outstand Perf Week
Ticker                                                            
VBFC          -  6.96      4.31      58.20%        1.45M     1.28%
HNR     Missing   NaN       NaN         NaN          NaN       NaN
FTBK    Missing   NaN       NaN         NaN          NaN       NaN
DONE


In [23]:
stocks.to_csv('stocks.csv')

# Scraping the sectors

In [15]:
stocks = pd.read_csv('stocks.csv', index_col=0)
features = list(stocks.columns.values)
# stocks['Sector'] = ''

In [6]:
def populateSector(stocks, tick, soup):
    table = soup.find('table', {"class":"fullview-title"})
    if not table == None:
        row = table.find_all('tr', recursive=False)[2]
        data = row.find_all('td', recursive=False)[0]
        anchor = data.find_all('a')[0]
        stocks.loc[tick, 'Sector'] = anchor.text
    else:
        stocks.loc[tick, 'Sector'] = 'Missing'
    return stocks

def getEmptyRowIndex(stocks):
    i = 0
    for _, row in stocks.iterrows():
        if isinstance(row['Sector'], float):
            break
        i += 1
    return i

In [8]:
global pv_count
# pv_count = 1176
i = 1
ind = getEmptyRowIndex(stocks)
for tick,_ in stocks.iterrows():
    if isinstance(stocks.loc[tick, 'Sector'], float):
        clear_output(wait=True)
        print(f'Requesting {tick} {pv_count}, {ind}. . . ', end=' ')
        soup = makeRequest(tick, pv_count)        
        print(f'Populating . . . ', end=' ')
        stocks = populateSector(stocks, tick, soup)
        pv_count += 1
        i += 1
        if i % 10 == 0:
            print(f'.%SAVING%.', end=' ')
            stocks.to_csv('stocks.csv')
        sec = randint(10,30)
        print(f'Relaxing for {sec} sec . . . ')
        print(stocks.iloc[ind - 1:ind + 4, 72])
        ind += 1
        print(len(stocks['Sector'].unique()))
        sleep(sec)
print(f'DONE')

Requesting FTBK 2798, 3489. . .  Populating . . .  Relaxing for 21 sec . . . 
Ticker
HNR     Missing
FTBK    Missing
Name: Sector, dtype: object
12
DONE


In [9]:
stocks.to_csv('stocks.csv')

In [5]:
stocks.head()

,Index,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y,Insider Trans,...,Payout,Avg Volume,Price,Recom,SMA20,SMA50,SMA200,Volume,Change,Sector
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,DJIA S&P500,36.75,3.29,0.07%,17.25B,-3.30%,2070.71B,31.12,19.84%,-3.09%,...,23.60%,161.82M,120.88,2.2,0.72%,14.41%,48.13%,"251,725,012",-8.01%,Technology
MSFT,DJIA S&P500,37.7,5.76,0.10%,7.58B,-4.10%,1627.71B,29.62,13.59%,-5.88%,...,34.20%,34.38M,217.30,1.7,0.66%,3.35%,21.59%,"57,706,830",-6.19%,Technology
AMZN,S&P 500,129.46,26.02,10.90%,500.00M,-0.94%,1686.09B,76.31,39.66%,-1.86%,...,0.00%,4.63M,3368.00,1.8,2.36%,7.90%,43.91%,"8,045,811",-4.63%,Consumer Cyclical
FB,S&P 500,37.14,7.84,0.66%,2.85B,-0.72%,828.11B,28.68,26.92%,-0.55%,...,0.00%,25.27M,291.12,1.8,6.03%,15.17%,35.94%,"31,485,584",-3.76%,Communication Services
GOOGL,S&P 500,36.85,44.22,0.14%,681.77M,0.06%,1102.88B,28.8,27.82%,0.00%,...,0.00%,1.75M,1629.51,1.8,3.85%,6.91%,17.21%,"3,093,648",-5.12%,Communication Services


In [8]:
stocks['Sector'].unique()

array(['Technology', 'Consumer Cyclical', 'Communication Services',
       'Healthcare', 'Financial', 'Consumer Defensive', 'Energy',
       'Utilities', 'Basic Materials', 'Industrials', 'Real Estate',
       'Missing'], dtype=object)

In [16]:
"""
basic materials => materials
consumer defensive => consumer staples
consumer cyclical => consumer discretionary
technology => information technology
"""
stocks['GICS Sector'] = stocks['Sector'].replace({'Basic Materials': 'Materials',
                                                  'Consumer Defensive': 'Consumer Staples',
                                                  'Consumer Cyclical': 'Consumer Discretionary',
                                                  'Technology': 'Information Technology'})

In [17]:
stocks['GICS Sector'].unique()

array(['Information Technology', 'Consumer Discretionary',
       'Communication Services', 'Healthcare', 'Financial',
       'Consumer Staples', 'Energy', 'Utilities', 'Materials',
       'Industrials', 'Real Estate', 'Missing'], dtype=object)

In [18]:
stocks = stocks.drop('Sector', axis=1)

In [19]:
stocks.head()

,Index,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y,Insider Trans,...,Payout,Avg Volume,Price,Recom,SMA20,SMA50,SMA200,Volume,Change,GICS Sector
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,DJIA S&P500,36.75,3.29,0.07%,17.25B,-3.30%,2070.71B,31.12,19.84%,-3.09%,...,23.60%,161.82M,120.88,2.2,0.72%,14.41%,48.13%,"251,725,012",-8.01%,Information Technology
MSFT,DJIA S&P500,37.7,5.76,0.10%,7.58B,-4.10%,1627.71B,29.62,13.59%,-5.88%,...,34.20%,34.38M,217.30,1.7,0.66%,3.35%,21.59%,"57,706,830",-6.19%,Information Technology
AMZN,S&P 500,129.46,26.02,10.90%,500.00M,-0.94%,1686.09B,76.31,39.66%,-1.86%,...,0.00%,4.63M,3368.00,1.8,2.36%,7.90%,43.91%,"8,045,811",-4.63%,Consumer Discretionary
FB,S&P 500,37.14,7.84,0.66%,2.85B,-0.72%,828.11B,28.68,26.92%,-0.55%,...,0.00%,25.27M,291.12,1.8,6.03%,15.17%,35.94%,"31,485,584",-3.76%,Communication Services
GOOGL,S&P 500,36.85,44.22,0.14%,681.77M,0.06%,1102.88B,28.8,27.82%,0.00%,...,0.00%,1.75M,1629.51,1.8,3.85%,6.91%,17.21%,"3,093,648",-5.12%,Communication Services


In [22]:
stocks_copy = stocks.copy()
print(len(stocks))
for tick,_ in stocks.iterrows():
    if stocks.loc[tick, 'Index'] == 'Missing' or stocks.loc[tick, 'Index'] == 'GICS Sector':
        stocks_copy = stocks.drop(tick, axis=0)
print(len(stocks_copy))


3490
3489


In [24]:
stocks_filtered = stocks[stocks['Index'] != 'Missing']

In [25]:
len(stocks_filtered)

3432

In [26]:
stocks_filtered = stocks_filtered[stocks_filtered['GICS Sector'] != 'Missing']
len(stocks_filtered)

3427

In [27]:
stocks_filtered.to_csv('./stocks.csv')